In [1]:
import numpy as np

In [ ]:
def create_qft_matrix(n):
    """
    Generates Quantum Fourier Transformation matrix 
    
    Parameters
    ----------
    n: int
        Number of qubits to be transformed
    
    Return
    ------
    
    qft: array_like
        QFT matrix
    """
    N = 2 ** n
    res = np.empty([states, states])
    
    w = croot(N, 1)
    
    for num in range(N):
        state = format(num, '0' + str(qubit_number) + 'b')
        
        
        res[:, num] = column[:, 0]
        
    
    

In [ ]:
def transfor_function_to_matrix(qubit_number, func):
    states = 2 ** qubit_number
    res = np.empty([states, states])
    
    for num in range(states):
        state = format(num, '0' + str(qubit_number) + 'b')
        X = state[:-1]
        y = str(int(state[-1]) ^ func(int(X, 2)))
        column = string_to_state(X+y)
        
        res[:, num] = column[:, 0]
    
    return res

In [13]:
def croot(n, k):
    if n<=0:
        return None
    return np.exp((2 * np.pi * 1j * k) / n)
    